# Corpus Preprocessing for Knowledge Graph 

## 1. Setup and Paths

In [113]:
# --- Imports ---

from pathlib import Path
import re

import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import unicodedata
from transliterate import translit
from unidecode import unidecode
from rapidfuzz import fuzz, process
import json
import ast

In [114]:
PROJECT_ROOT = Path("/Users/eugenia/Desktop/thesis/magic_tagger")

INDEX_DIR = PROJECT_ROOT / "data" / "index"

## 2. Load Source Tables

In [115]:
df = pd.read_excel(INDEX_DIR / "corpus_a_index_clean_atu.xlsx")

print("Shape:", df.shape)
print("\nColumns:")
print(list(df.columns))

display(df.head())

print("\nDtypes:")
print(df.info())

Shape: (112, 36)

Columns:
['tale_id', 'collection', 'volume_no', 'source_ref', 'digital_carrier', 'rights_status', 'narrator', 'collector_1', 'collector_2', 'collector_3', 'collector_4', 'collector_5', 'narrator_school', 'recording_parish', 'recording_place', 'narrator_origin_parish', 'narrator_origin_place', 'recorded_date_start', 'recorded_date_end', 'content_description', 'genre_1', 'genre_2', 'genre_3', 'subgenre', 'folklore_category', 'local_type_scheme', 'local_type', 'type_code_1', 'type_code_2', 'type_code_3', 'type_code_4', 'mapping_status', 'mapping_relation', 'mapping_date', 'mapping_by', 'gold_status']


,tale_id,collection,volume_no,source_ref,digital_carrier,rights_status,narrator,collector_1,collector_2,collector_3,...,local_type,type_code_1,type_code_2,type_code_3,type_code_4,mapping_status,mapping_relation,mapping_date,mapping_by,gold_status
0,era_vene_2_605_4,"ERA, Vene",2,"ERA, Vene 2, 605/21 (4)",scan_only,open,"Nimi: Денисиха, Vanus/Sünniaasta: 75 л.",Федор Коняев,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,missing,exact,20.12.2025,Evgeniia Vdovichenko,missing
1,era_vene_5_167_9,"ERA, Vene",5,"ERA, Vene 5, 167/9 (9)",scan_only,open,NaN,Nina Valkevitš,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,missing,exact,20.12.2025,Evgeniia Vdovichenko,missing
2,era_vene_6_379_6,"ERA, Vene",6,"ERA, Vene 6, 379/81",scan_only,open,"Nimi: Lewonty Borhov, Vanus/Sünniaasta: 63 a.",M. Sokolov,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,missing,exact,20.12.2025,Evgeniia Vdovichenko,missing
3,era_vene_7_125_1,"ERA, Vene",7,"ERA, Vene 7, 125/8 (1)",scan_only,open,"Nimi: Александр Кельнер, Märkus: слышал от баб...",Александр Кельнер,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,missing,exact,20.12.2025,Evgeniia Vdovichenko,missing
4,rkm_vene_3_257_116,"RKM, Vene",3,"RKM, Vene 3, 257/62 (116)",scan_only,open,"Nimi: Анастасия Кодар, Vanus/Sünniaasta: 1909,...",Ольга Гильдебрандт,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,missing,exact,20.12.2025,Evgeniia Vdovichenko,missing



Dtypes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   tale_id                 112 non-null    object        
 1   collection              112 non-null    object        
 2   volume_no               112 non-null    int64         
 3   source_ref              112 non-null    object        
 4   digital_carrier         112 non-null    object        
 5   rights_status           112 non-null    object        
 6   narrator                110 non-null    object        
 7   collector_1             112 non-null    object        
 8   collector_2             21 non-null     object        
 9   collector_3             10 non-null     object        
 10  collector_4             8 non-null      object        
 11  collector_5             2 non-null      object        
 12  narrator_school         14 non-null     o

## 3. Choose columns for the first model draft

In [116]:
# Select the KG-relevant columns (v1 slice)

KG_COLS_V1 = [
    "tale_id",
    "collection",
    "volume_no",
    "source_ref",
    "rights_status",
    "narrator",
    "collector_1",
    "collector_2",
    "collector_3",
    "collector_4",
    "collector_5",
    "recording_parish",
    "recording_place",
    "recorded_date_start",
    "content_description",
    "local_type_scheme",
    "local_type",
    "type_code_1",
    "type_code_2",
    "type_code_3",
    "type_code_4",
]

# Keep only existing columns (safe if some are missing)
cols_present = [c for c in KG_COLS_V1 if c in df.columns]

df_kg = df.loc[:, cols_present].copy()

df_kg = df_kg.reindex(columns=cols_present)

df_kg.head()


,tale_id,collection,volume_no,source_ref,rights_status,narrator,collector_1,collector_2,collector_3,collector_4,...,recording_parish,recording_place,recorded_date_start,content_description,local_type_scheme,local_type,type_code_1,type_code_2,type_code_3,type_code_4
0,era_vene_2_605_4,"ERA, Vene",2,"ERA, Vene 2, 605/21 (4)",open,"Nimi: Денисиха, Vanus/Sünniaasta: 75 л.",Федор Коняев,NaN,NaN,NaN,...,Kodavere khk.,"Peipsiäärne v., Nina (Нос) k.",1928,Сказка о Коле. [К царю приходят преступник и з...,NaN,NaN,NaN,NaN,NaN,NaN
1,era_vene_5_167_9,"ERA, Vene",5,"ERA, Vene 5, 167/9 (9)",open,NaN,Nina Valkevitš,NaN,NaN,NaN,...,Petserimaa,Irboska v.,1934,[После десяти лет петух сносит яйцо. Мужик и с...,NaN,NaN,NaN,NaN,NaN,NaN
2,era_vene_6_379_6,"ERA, Vene",6,"ERA, Vene 6, 379/81",open,"Nimi: Lewonty Borhov, Vanus/Sünniaasta: 63 a.",M. Sokolov,NaN,NaN,NaN,...,Petserimaa,"Irboska v., Vastsõ k.",1935,"Про Брюса и про шишка : ня сказка, а истинно с...",NaN,NaN,NaN,NaN,NaN,NaN
3,era_vene_7_125_1,"ERA, Vene",7,"ERA, Vene 7, 125/8 (1)",open,"Nimi: Александр Кельнер, Märkus: слышал от баб...",Александр Кельнер,NaN,NaN,NaN,...,Tallinna linn,NaN,1935,Сказка о золотом петушке. [Пересказ сказки А. ...,NaN,NaN,NaN,NaN,NaN,NaN
4,rkm_vene_3_257_116,"RKM, Vene",3,"RKM, Vene 3, 257/62 (116)",open,"Nimi: Анастасия Кодар, Vanus/Sünniaasta: 1909,...",Ольга Гильдебрандт,NaN,NaN,NaN,...,Tartu linn,NaN,1949,"[Дочь зовёт супряженок прясть, чтобы не остава...",NaN,NaN,NaN,NaN,NaN,NaN


## 4. Change date datatypes

In [117]:
#  Normalize recorded_date_start to datetime64[ns] (usable type)
def normalize_recorded_date_start(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA, "None": pd.NA})

    # Year-only detection
    year_only = s.str.fullmatch(r"\d{4}", na=False)

    out = pd.Series(pd.NaT, index=s.index, dtype="datetime64[ns]")

    # Parse year-only as Jan 1st of that year (keeps datetime dtype)
    if year_only.any():
        out.loc[year_only] = pd.to_datetime(s.loc[year_only] + "-01-01", errors="coerce", format="%Y-%m-%d")

    # Parse the rest:
    rest = ~year_only & s.notna()
    if rest.any():
        # Try US-style M/D/YYYY first
        parsed_us = pd.to_datetime(s.loc[rest], errors="coerce", dayfirst=False)
        out.loc[rest] = parsed_us

        # If some still failed, try ISO explicitly (sometimes helps)
        failed = rest & out.isna()
        if failed.any():
            out.loc[failed] = pd.to_datetime(s.loc[failed], errors="coerce", format="%Y-%m-%d")

    return out

# Apply (drop recorded_date_end entirely as requested)
if "recorded_date_end" in df_kg.columns:
    df_kg = df_kg.drop(columns=["recorded_date_end"], errors="ignore")

df_kg["recorded_date_start_dt"] = normalize_recorded_date_start(df_kg["recorded_date_start"])

# (Optional) if you want the column name to stay the same:
df_kg["recorded_date_start"] = df_kg["recorded_date_start_dt"]
df_kg = df_kg.drop(columns=["recorded_date_start_dt"])

# Quick QC
print("Parsed start dates:", df_kg["recorded_date_start"].notna().sum(), "/", len(df_kg))
print("Examples of unparsed raw values:")

Parsed start dates: 112 / 112
Examples of unparsed raw values:


## 5. Change collection and volume writings

In [118]:
# --- helpers -------------------------------------------------

def norm_token(s: pd.Series) -> pd.Series:
    """
    Canonical token for IDs:
    - remove commas
    - trim
    - collapse whitespace
    - spaces -> underscores
    - lowercase
    """
    return (
        s.astype("string")
         .str.replace(",", "", regex=False)
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
         .str.replace(" ", "_", regex=False)
         .str.lower()
    )

def norm_text(s: pd.Series) -> pd.Series:
    """
    Lightweight normalization for free-text fields:
    - trim
    - collapse whitespace
    (keeps original case/punctuation)
    """
    return (
        s.astype("string")
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
    )

In [119]:
# --- normalize ID-like columns ----------------------------

# collection: "ERA, Vene" / "ERA_Vene" -> "era_vene"
df_kg["collection"] = norm_token(df_kg["collection"])

# tale_id: enforce canonical lowercase (and trim)
df_kg["tale_id"] = (
    df_kg["tale_id"]
      .astype("string")
      .str.strip()
      .str.lower()
)

# volume_no: ensure integer dtype (nullable)
df_kg["volume_no"] = pd.to_numeric(df_kg["volume_no"], errors="coerce").astype("Int64")

# derived id: volume_id (canonical)
df_kg["volume_id"] = df_kg["collection"] + "_" + df_kg["volume_no"].astype("string")

In [120]:
df_kg.head()

,tale_id,collection,volume_no,source_ref,rights_status,narrator,collector_1,collector_2,collector_3,collector_4,...,recording_place,recorded_date_start,content_description,local_type_scheme,local_type,type_code_1,type_code_2,type_code_3,type_code_4,volume_id
0,era_vene_2_605_4,era_vene,2,"ERA, Vene 2, 605/21 (4)",open,"Nimi: Денисиха, Vanus/Sünniaasta: 75 л.",Федор Коняев,NaN,NaN,NaN,...,"Peipsiäärne v., Nina (Нос) k.",1928-01-01,Сказка о Коле. [К царю приходят преступник и з...,NaN,NaN,NaN,NaN,NaN,NaN,era_vene_2
1,era_vene_5_167_9,era_vene,5,"ERA, Vene 5, 167/9 (9)",open,NaN,Nina Valkevitš,NaN,NaN,NaN,...,Irboska v.,1934-01-01,[После десяти лет петух сносит яйцо. Мужик и с...,NaN,NaN,NaN,NaN,NaN,NaN,era_vene_5
2,era_vene_6_379_6,era_vene,6,"ERA, Vene 6, 379/81",open,"Nimi: Lewonty Borhov, Vanus/Sünniaasta: 63 a.",M. Sokolov,NaN,NaN,NaN,...,"Irboska v., Vastsõ k.",1935-01-01,"Про Брюса и про шишка : ня сказка, а истинно с...",NaN,NaN,NaN,NaN,NaN,NaN,era_vene_6
3,era_vene_7_125_1,era_vene,7,"ERA, Vene 7, 125/8 (1)",open,"Nimi: Александр Кельнер, Märkus: слышал от баб...",Александр Кельнер,NaN,NaN,NaN,...,NaN,1935-01-01,Сказка о золотом петушке. [Пересказ сказки А. ...,NaN,NaN,NaN,NaN,NaN,NaN,era_vene_7
4,rkm_vene_3_257_116,rkm_vene,3,"RKM, Vene 3, 257/62 (116)",open,"Nimi: Анастасия Кодар, Vanus/Sünniaasta: 1909,...",Ольга Гильдебрандт,NaN,NaN,NaN,...,NaN,1949-01-01,"[Дочь зовёт супряженок прясть, чтобы не остава...",NaN,NaN,NaN,NaN,NaN,NaN,rkm_vene_3


## 6. Check types columns

In [121]:

CYR_TO_LAT = str.maketrans({
    "А": "A", "В": "B", "Е": "E", "К": "K", "М": "M", "Н": "H",
    "О": "O", "Р": "P", "С": "C", "Т": "T", "Х": "X", "У": "Y",
    "а": "a", "в": "b", "е": "e", "к": "k", "м": "m", "н": "h",
    "о": "o", "р": "p", "с": "c", "т": "t", "х": "x", "у": "y",
})


ATU_RE = re.compile(r"^\s*(\d{1,4})\s*([A-Za-z])?\s*(\*?)\s*$")

def normalize_atu_code(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return pd.NA
    s = str(x).strip()
    if not s:
        return pd.NA

    
    s = s.translate(CYR_TO_LAT)

    
    s = re.sub(r"[\s_\-]+", "", s)

    
    m = ATU_RE.match(s)
    if not m:
        
        return s

    num, letter, star = m.groups()
    letter = (letter or "").upper()
    return f"{num}{letter}{star}"


In [122]:
CODE_COLS = ["type_code_1", "type_code_2", "type_code_3", "type_code_4", "local_type"]

df_kg = df_kg.copy()
for c in CODE_COLS:
    if c in df_kg.columns:
        df_kg[c] = df_kg[c].apply(normalize_atu_code)


# ----------------------------
# Compare df (raw) vs df_kg (cleaned) by tale_id
# ----------------------------
# take only needed columns from df (raw)
cols_for_compare = ["tale_id"] + [c for c in CODE_COLS if c in df.columns and c in df_kg.columns]

raw_codes = df[cols_for_compare].copy()
clean_codes = df_kg[cols_for_compare].copy()

# ensure string dtype for stable comparison
for c in CODE_COLS:
    if c in raw_codes.columns:
        raw_codes[c] = raw_codes[c].astype("string")
        clean_codes[c] = clean_codes[c].astype("string")

merged = raw_codes.merge(
    clean_codes,
    on="tale_id",
    suffixes=("_raw", "_clean"),
    how="inner"  # assumes df_kg tale_id subset of df
)

# build a tidy diff table
diff_frames = []
for c in CODE_COLS:
    if f"{c}_raw" not in merged.columns:
        continue
    mask = merged[f"{c}_raw"].fillna("") != merged[f"{c}_clean"].fillna("")
    if mask.any():
        diff_frames.append(
            merged.loc[mask, ["tale_id", f"{c}_raw", f"{c}_clean"]].rename(
                columns={f"{c}_raw": "raw", f"{c}_clean": "clean"}
            ).assign(column=c)
        )

diff = pd.concat(diff_frames, ignore_index=True) if diff_frames else pd.DataFrame(columns=["tale_id", "column", "raw", "clean"])

print("Total changed cells:", len(diff))
display(diff.sort_values(["column", "tale_id"]).head(50))


Total changed cells: 7


,tale_id,raw,clean,column
5,era_vene_14_451_7,650С*,650C*,local_type
4,era_vene_15_433_1,650В*,650B*,local_type
6,era_vene_7_89_1,480E*,480E*,local_type
0,era_vene_15_433_1,650A,650A,type_code_1
1,era_vene_8_308_277,326В*,326B*,type_code_1
2,era_vene_13_318_27,302С*,302C*,type_code_3
3,era_vene_13_318_27,556А*,556A*,type_code_4


In [123]:
def has_latin_letter(s):
    if s is None or pd.isna(s):
        return False
    return re.search(r"[A-Za-z]", str(s)) is not None

def letter_dropped(raw, clean):
    if raw is None or pd.isna(raw) or clean is None or pd.isna(clean):
        return False
    raw_norm = str(raw).translate(CYR_TO_LAT)
    return has_latin_letter(raw_norm) and (not has_latin_letter(clean))

dropped_frames = []
for c in CODE_COLS:
    if f"{c}_raw" not in merged.columns:
        continue
    mask = merged.apply(lambda r: letter_dropped(r[f"{c}_raw"], r[f"{c}_clean"]), axis=1)
    if mask.any():
        dropped_frames.append(
            merged.loc[mask, ["tale_id", f"{c}_raw", f"{c}_clean"]]
            .rename(columns={f"{c}_raw": "raw", f"{c}_clean": "clean"})
            .assign(column=c)
        )

dropped = pd.concat(dropped_frames, ignore_index=True) if dropped_frames else pd.DataFrame(columns=["tale_id", "column", "raw", "clean"])

print("\nLetter-dropped cases:", len(dropped))
display(dropped.sort_values(["column", "tale_id"]))


Letter-dropped cases: 0


,tale_id,column,raw,clean


In [124]:
def to_list_str(x) -> list[str]:

    if x is None or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and not x.strip()):
        return []
    if isinstance(x, list):
        return [str(t).strip() for t in x if str(t).strip()]

    s = str(x).strip()

    # 1) 
    try:
        v = json.loads(s)
        if isinstance(v, list):
            return [str(t).strip() for t in v if str(t).strip()]
        return [str(v).strip()] if str(v).strip() else []
    except Exception:
        pass


    try:
        v = ast.literal_eval(s)
        if isinstance(v, list):
            return [str(t).strip() for t in v if str(t).strip()]
    except Exception:
        pass


    parts = re.split(r"[,\;\|/]+|\s{2,}", s)
    return [p.strip() for p in parts if p and p.strip()]

def merge_row_codes(row) -> list[str]:
    items: list[str] = []
    for col in CODE_COLS:
        for t in to_list_str(row.get(col)):
            items.append(str(t))

    out = []
    seen = set()
    for t in items:
        t = t.strip()
        if not t or t.lower() == "nan":
            continue
        if t not in seen:
            seen.add(t)
            out.append(t)
    return out

df_kg["atu_codes"] = df_kg.apply(merge_row_codes, axis=1)

## 7.  Place labels and multilingual normalization

`recording_parish` and `recording_place` occur in the archival language (Estonian/Russian).
For FAIR interoperability, the published KG does **not overwrite** source strings.
Instead, each distinct place string is mapped to a `crm:E53_Place` resource with:

- the source form as `skos:prefLabel` (language-tagged when known),
- the English normalized form as `skos:altLabel` (`@en`).

Tales link to places via `dcterms:spatial` (object property to the Place URI).
The English mapping is maintained as a separate, versioned lookup table and can evolve without losing the original archival labels.


In [125]:
map_path = PROJECT_ROOT / "data" / "processed" / "index_with_places.csv"
places_map = pd.read_csv(map_path, dtype={"tale_id": "string"})

# normalize join key
df_kg["tale_id"] = df_kg["tale_id"].astype("string").str.strip()
places_map["tale_id"] = places_map["tale_id"].astype("string").str.strip()

cols_to_add = [
    "tale_id",
    "recording_parish_english",
    "recording_place_english",
    "region_english",
    "country_english",
]

missing = [c for c in cols_to_add if c not in places_map.columns]
if missing:
    raise ValueError(f"Missing columns in mapping file: {missing}")

# (optional but recommended) drop duplicate keys in mapping, keep first
dups = places_map["tale_id"].duplicated(keep=False)
if dups.any():
    print("WARNING: duplicate tale_id in mapping file. Keeping first occurrence for each tale_id.")
    places_map = places_map.drop_duplicates(subset=["tale_id"], keep="first")

# merge into df_kg
df_kg = df_kg.merge(
    places_map[cols_to_add],
    on="tale_id",
    how="left",
)

# quick checks
print("Coverage (non-null rate) of added English columns:")
print(df_kg[cols_to_add[1:]].notna().mean().sort_values(ascending=False))

Coverage (non-null rate) of added English columns:
recording_parish_english    1.000000
region_english              1.000000
country_english             1.000000
recording_place_english     0.901786
dtype: float64


In [126]:
print("\nUnmatched examples (first 20):")
unmatched = df_kg[df_kg["recording_parish_english"].isna() & df_kg["recording_parish"].notna()][
    ["tale_id", "recording_parish", "recording_place"]
]
print(unmatched.head(20))


Unmatched examples (first 20):
Empty DataFrame
Columns: [tale_id, recording_parish, recording_place]
Index: []


In [127]:
df_kg.head()

,tale_id,collection,volume_no,source_ref,rights_status,narrator,collector_1,collector_2,collector_3,collector_4,...,type_code_1,type_code_2,type_code_3,type_code_4,volume_id,atu_codes,recording_parish_english,recording_place_english,region_english,country_english
0,era_vene_2_605_4,era_vene,2,"ERA, Vene 2, 605/21 (4)",open,"Nimi: Денисиха, Vanus/Sünniaasta: 75 л.",Федор Коняев,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,era_vene_2,[<NA>],Kodavere parish,Nina village,Tartu County,Estonia
1,era_vene_5_167_9,era_vene,5,"ERA, Vene 5, 167/9 (9)",open,NaN,Nina Valkevitš,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,era_vene_5,[<NA>],Petseri County,Izborsk,Pskov Oblast,Russia
2,era_vene_6_379_6,era_vene,6,"ERA, Vene 6, 379/81",open,"Nimi: Lewonty Borhov, Vanus/Sünniaasta: 63 a.",M. Sokolov,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,era_vene_6,[<NA>],Petseri County,Vasttsy village,Pskov Oblast,Russia
3,era_vene_7_125_1,era_vene,7,"ERA, Vene 7, 125/8 (1)",open,"Nimi: Александр Кельнер, Märkus: слышал от баб...",Александр Кельнер,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,era_vene_7,[<NA>],Tallinn,NaN,Harju County,Estonia
4,rkm_vene_3_257_116,rkm_vene,3,"RKM, Vene 3, 257/62 (116)",open,"Nimi: Анастасия Кодар, Vanus/Sünniaasta: 1909,...",Ольга Гильдебрандт,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,rkm_vene_3,[<NA>],Tartu,NaN,Tartu County,Estonia


## 8. Normalizing agents columns 

In [128]:
# nar010, nar011, nar_011, nar-011, NAR011a (если вдруг есть суффиксы)
NAR_ID_RE = re.compile(r"\b(nar[_-]?\d{2,5}[a-z]?)\b", re.IGNORECASE)

FIELD_RE = re.compile(r"(Nimi|Vanus/Sünniaasta|Märkus)\s*:\s*", flags=re.IGNORECASE)

def clean_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def extract_field(text: str, field_name: str) -> str | None:
    """
    Extracts '<field_name>: value' until next ', <Something>:' or end.
    """
    if not isinstance(text, str) or not text.strip():
        return None
    pattern = re.compile(rf"{re.escape(field_name)}\s*:\s*(.*?)(?=,\s*[^,]{{1,40}}?:|$)", re.IGNORECASE)
    m = pattern.search(text)
    if not m:
        return None
    return clean_spaces(m.group(1)).strip(",; ")

def parse_age_or_birth_year(s: str) -> tuple[int | None, int | None]:
    if not isinstance(s, str) or not s.strip():
        return (None, None)
    s = s.strip()

    # 4-digit year
    m_year = re.search(r"\b(1[7-9]\d{2}|20\d{2})\b", s)
    if m_year:
        y = int(m_year.group(1))
        if 1700 <= y <= 2026:
            return (None, y)

    # age number
    m_age = re.search(r"\b(\d{1,3})\b", s)
    if m_age:
        age = int(m_age.group(1))
        if 0 < age < 130:
            return (age, None)

    return (None, None)

def find_nar_id(text: str | None) -> str | None:
    """
    Finds nar id anywhere in string. Returns canonical form: nar + digits (+ optional suffix), lowercase, no separators.
    Examples:
      'nar_010' -> 'nar010'
      'Nimi: nar-011' -> 'nar011'
    """
    if not isinstance(text, str) or not text.strip():
        return None
    m = NAR_ID_RE.search(text)
    if not m:
        return None
    raw = m.group(1).lower()
    raw = raw.replace("_", "").replace("-", "")
    return raw

def make_slug_en(label: str) -> str:
    s = unidecode(label)
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9]+", "-", s)
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s

def parse_narrator_cell(cell: str) -> dict:
    """
    Produces:
      narrator_name_raw, narrator_note_raw, narrator_age, narrator_birth_year,
      narrator_label_en, narrator_person_id
    Key rule:
      - if nar### appears either as the whole cell OR inside Nimi OR anywhere in raw cell:
        narrator_label_en = nar### (canonical), narrator_person_id = nar###
    """
    if not isinstance(cell, str) or not cell.strip():
        return {
            "narrator_name_raw": None,
            "narrator_note_raw": None,
            "narrator_age": None,
            "narrator_birth_year": None,
            "narrator_label_en": None,
            "narrator_person_id": None,
        }

    cell = cell.strip()

    # Parse fields (if present)
    name = extract_field(cell, "Nimi")
    age_or_year = extract_field(cell, "Vanus/Sünniaasta")
    note = extract_field(cell, "Märkus")

    # 1) Detect nar### robustly (priority):
    nar = (
        find_nar_id(name)      # nar as name
        or find_nar_id(cell)   # nar appears anywhere in raw
    )
    if nar:
        age, birth_year = parse_age_or_birth_year(age_or_year) if age_or_year else (None, None)
        return {
            "narrator_name_raw": name if name else nar,
            "narrator_note_raw": note,
            "narrator_age": age,
            "narrator_birth_year": birth_year,
            "narrator_label_en": nar,     # always set
            "narrator_person_id": nar,    # always set
        }

    # 2) Normal case: name exists -> transliterate best-effort with unidecode
    base_label = name if name else cell
    label_en = clean_spaces(unidecode(base_label))
    person_id = make_slug_en(name if name else label_en)

    age, birth_year = parse_age_or_birth_year(age_or_year) if age_or_year else (None, None)

    return {
        "narrator_name_raw": name if name else cell,
        "narrator_note_raw": note,
        "narrator_age": age,
        "narrator_birth_year": birth_year,
        "narrator_label_en": label_en,
        "narrator_person_id": person_id,
    }

In [129]:
# ----------------------------
# Apply to df_kg
# ----------------------------

def enrich_narrators(df_kg: pd.DataFrame, col: str = "narrator") -> pd.DataFrame:
    out = df_kg.copy()
    parsed = out[col].apply(parse_narrator_cell).apply(pd.Series)
    for c in parsed.columns:
        out[c] = parsed[c]
    return out

df_kg = enrich_narrators(df_kg, col="narrator")

In [130]:
df_kg.head()

,tale_id,collection,volume_no,source_ref,rights_status,narrator,collector_1,collector_2,collector_3,collector_4,...,recording_parish_english,recording_place_english,region_english,country_english,narrator_name_raw,narrator_note_raw,narrator_age,narrator_birth_year,narrator_label_en,narrator_person_id
0,era_vene_2_605_4,era_vene,2,"ERA, Vene 2, 605/21 (4)",open,"Nimi: Денисиха, Vanus/Sünniaasta: 75 л.",Федор Коняев,NaN,NaN,NaN,...,Kodavere parish,Nina village,Tartu County,Estonia,Денисиха,None,75.0,NaN,Denisikha,denisikha
1,era_vene_5_167_9,era_vene,5,"ERA, Vene 5, 167/9 (9)",open,NaN,Nina Valkevitš,NaN,NaN,NaN,...,Petseri County,Izborsk,Pskov Oblast,Russia,None,None,NaN,NaN,None,None
2,era_vene_6_379_6,era_vene,6,"ERA, Vene 6, 379/81",open,"Nimi: Lewonty Borhov, Vanus/Sünniaasta: 63 a.",M. Sokolov,NaN,NaN,NaN,...,Petseri County,Vasttsy village,Pskov Oblast,Russia,Lewonty Borhov,None,63.0,NaN,Lewonty Borhov,lewonty-borhov
3,era_vene_7_125_1,era_vene,7,"ERA, Vene 7, 125/8 (1)",open,"Nimi: Александр Кельнер, Märkus: слышал от баб...",Александр Кельнер,NaN,NaN,NaN,...,Tallinn,NaN,Harju County,Estonia,Александр Кельнер,слышал от бабушки,NaN,NaN,Aleksandr Kel'ner,aleksandr-kel-ner
4,rkm_vene_3_257_116,rkm_vene,3,"RKM, Vene 3, 257/62 (116)",open,"Nimi: Анастасия Кодар, Vanus/Sünniaasta: 1909,...",Ольга Гильдебрандт,NaN,NaN,NaN,...,Tartu,NaN,Tartu County,Estonia,Анастасия Кодар,слышала от отца,NaN,1909.0,Anastasiia Kodar,anastasiia-kodar


In [131]:
arr = df_kg["narrator_person_id"].unique()

In [132]:
ids = list(pd.Series(arr).dropna().astype(str).unique())  # arr = твой numpy array
ids = [x.strip() for x in ids if x.strip()]

# ---- 2) нормализация для поиска "почти одинаковых" ----
VOWELS = set("aeiouy")

def canon_key(s: str) -> str:
    
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9]+", "", s)          # убрали дефисы и мусор
    # remove vowels
    s2 = "".join(ch for ch in s if ch not in VOWELS)
    # compress repeats
    s3 = re.sub(r"(.)\1+", r"\1", s2)
    return s3

def token_key(s: str) -> str:
    
    s = s.lower().strip()
    toks = re.split(r"[-_]+", s)
    toks = [re.sub(r"[^a-z0-9]+", "", t) for t in toks]
    toks = [t for t in toks if t]
    return "-".join(toks)


In [133]:
# ---- 3) быстрые "точные" подозрения по canon_key ----
df = pd.DataFrame({"id": ids})
df["token_key"] = df["id"].apply(token_key)
df["canon_key"] = df["id"].apply(canon_key)

susp_exact = (
    df.groupby("canon_key")
      .agg(ids=("id", lambda s: sorted(set(s))), n=("id", "nunique"))
      .reset_index()
)
susp_exact = susp_exact[susp_exact["n"] > 1].sort_values("n", ascending=False)

print("=== Candidates by canon_key (strong heuristic) ===")
display(susp_exact.head(50))

=== Candidates by canon_key (strong heuristic) ===


,canon_key,ids,n
28,nmkhn,"[an-mukhina, anna-mukhina, nina-mukhina]",3
29,nr01,"[nar001, nar011]",2


In [134]:

choices = df["token_key"].tolist()

pairs = []
for i, row in df.iterrows():
    q = row["token_key"]
   
    for m, score, _ in process.extract(q, choices, scorer=fuzz.ratio, limit=8):
        if m == q:
            continue
        if score >= 90:  
            a = row["id"]
            b = df.loc[df["token_key"] == m, "id"].iloc[0]
            if a == b:
                continue
            x, y = sorted([a, b])
            pairs.append((x, y, score))

pairs_df = pd.DataFrame(sorted(set(pairs), key=lambda x: (-x[2], x[0], x[1])),
                        columns=["id_a", "id_b", "similarity"])

print("=== Candidates by fuzzy similarity (>=90) ===")
display(pairs_df.head(80))

=== Candidates by fuzzy similarity (>=90) ===


,id_a,id_b,similarity
0,anna-mukhina,nina-mukhina,91.666667
1,an-mukhina,anna-mukhina,90.909091


In [135]:
shortlist = pd.concat([
    susp_exact.assign(method="canon_key")[["method","ids","n"]],
    pairs_df.assign(method="fuzzy_pair")[["method","id_a","id_b","similarity"]]
], ignore_index=True)

shortlist

,method,ids,n,id_a,id_b,similarity
0,canon_key,"[an-mukhina, anna-mukhina, nina-mukhina]",3.0,NaN,NaN,NaN
1,canon_key,"[nar001, nar011]",2.0,NaN,NaN,NaN
2,fuzzy_pair,NaN,NaN,anna-mukhina,nina-mukhina,91.666667
3,fuzzy_pair,NaN,NaN,an-mukhina,anna-mukhina,90.909091


In [136]:
arr

array(['denisikha', None, 'lewonty-borhov', 'aleksandr-kel-ner',
       'anastasiia-kodar', 'nar018', 'nar006', 'nar002', 'nar004',
       'sergei-egorovich-domashkin', 'evgenii-mosharov',
       'georgii-samolevskii', 'vassa-denisovna-gunina', 'nar010',
       'nar016', 'anna-mukhina', 'an-mukhina', 'neizvestno',
       'anna-khalat', 'nina-mukhina', 'dariia-lipina',
       'nastasja-kropetskaja', 'z-vasil-eva-suvorova', 'ivan-mishin',
       'zoia-aleksandrovna-iupatova', 'anna-mikhailovna-seniushkina',
       'andrei-nikhandrovich-lik', 'ivan-petrovich-somov',
       'ivan-petrovich-kirshinskii', 'maksim-tverdov', 'anatolii-semenov',
       'nar015', 'nar009', 'nar013', 'nar007', 'nar001', 'nar003',
       'natal-ia-alekseevna-pagareva', 'anastasiia-dorofeevna-topkina',
       'iraida-georgievna-topkina', 'sergei-dimitrievich-tsvetkov',
       'e-protasova', 's-minusov', 'starukha-guseva', 'fedor-minushkin',
       'michail-dragunin', 'nikolai-grecman', 'a-billio',
       'oleg-rium

In [137]:
MERGE_MAP = {
    # your confirmed merges
    "an-mukhina": "anna-mukhina",
    "leon-borkhov": "lewonty-borhov",
    "aleksandr-kel-ner": "aleksandr-kelner",

    # normalize "broken" transliteration / hyphen splits
    "z-vasil-eva-suvorova": "z-vasilieva-suvorova",
    "natal-ia-alekseevna-pagareva": "natalia-alekseevna-pagareva",
    "tat-iana-semukhina": "tatiana-semukhina",
    "irin-ia-sal-nikova": "irina-salnikova",
    "ivan-mal-tsev": "ivan-maltsev",
    "avdot-ia-suvorova": "avdotia-suvorova",
    "dar-ia-rosolova": "dariia-rosolova",
    "avdot-ia-ershova": "avdotia-ershova",
    "dar-ia-zalavskaia": "dariia-zalavskaia",
    "aksin-ia": "aksinia",
    "anna-terent-evna-gunina": "anna-terentevna-gunina",
    "ol-ga-gil-debrandt": "olga-gildebrandt",
    'natal-ia-alekseevna-pagareva': "natalia-alekseevna-pagareva"

}


In [138]:
df_kg["narrator_person_id_canon"] = df_kg["narrator_person_id"].replace(MERGE_MAP)

changed = df_kg.loc[
    df_kg["narrator_person_id"].notna()
    & (df_kg["narrator_person_id"] != df_kg["narrator_person_id_canon"]),
    ["tale_id", "narrator", "narrator_person_id", "narrator_person_id_canon"]
].sort_values(["narrator_person_id", "tale_id"])

changed

,tale_id,narrator,narrator_person_id,narrator_person_id_canon
92,era_vene_13_37_5,"Nimi: Аксинья, Vanus/Sünniaasta: 72 л., Märkus...",aksin-ia,aksinia
3,era_vene_7_125_1,"Nimi: Александр Кельнер, Märkus: слышал от баб...",aleksandr-kel-ner,aleksandr-kelner
16,era_vene_2_161_14,Nimi: Ан. Мухина,an-mukhina,anna-mukhina
99,rkm_vene_1_236_5,"Nimi: Анна Терентьевна Гунина, Vanus/Sünniaast...",anna-terent-evna-gunina,anna-terentevna-gunina
80,era_vene_13_15_1,"Nimi: Авдотья Ершова, Vanus/Sünniaasta: 76 л.,...",avdot-ia-ershova,avdotia-ershova
83,era_vene_13_19_2,"Nimi: Авдотья Ершова, Vanus/Sünniaasta: 76 л.,...",avdot-ia-ershova,avdotia-ershova
73,era_vene_12_541_1,"Nimi: Авдотья Суворова, Vanus/Sünniaasta: 60 лет",avdot-ia-suvorova,avdotia-suvorova
79,era_vene_13_137_16,"Nimi: Дарья Росолова, Vanus/Sünniaasta: 73 л.,...",dar-ia-rosolova,dariia-rosolova
81,era_vene_13_151_17,"Nimi: Дарья Росолова, Vanus/Sünniaasta: 73 л.,...",dar-ia-rosolova,dariia-rosolova
87,era_vene_13_28_3,"Nimi: Дарья Залавская, Vanus/Sünniaasta: 60 л.",dar-ia-zalavskaia,dariia-zalavskaia


In [139]:
dups_after = (
    df_kg.dropna(subset=["narrator_person_id_canon"])
        .groupby("narrator_person_id_canon")["narrator_person_id"]
        .nunique()
        .sort_values(ascending=False)
)

dups_after[dups_after > 1]

narrator_person_id_canon
lewonty-borhov    2
anna-mukhina      2
Name: narrator_person_id, dtype: int64

In [140]:
# collector ids: col001, col_001, col-001, COL010, col012a
COL_ID_RE = re.compile(r"\b(col[_-]?\d{2,5}[a-z]?)\b", re.IGNORECASE)

def canon_collector_code(code: str) -> str:
    """
    Canonicalize collector code:
    - lowercase
    - normalize separators: col_010 / col-010 -> col010
    """
    c = code.strip().lower()
    c = c.replace("_", "").replace("-", "")
    return c


In [141]:
def parse_collector_cell(value: object) -> dict | None:
    """
    Parse collector cell.
    - If it contains a collector code (col001/col_001/col-001), use it as person_id and label_en.
    - Otherwise fall back to your name slugging (make_person_slug).

    Output keys:
    - person_id
    - label_en
    - raw
    - (optional) name_raw
    """
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return None

    s = str(value).strip()
    if not s:
        return None

    # 1) hard code match (collector anonymous/id)
    m = COL_ID_RE.search(s)
    if m:
        pid = canon_collector_code(m.group(1))
        return {"person_id": pid, "label_en": pid, "raw": s}

    # 2) otherwise: treat as name (keep it very lightweight)
    # collectors are usually plain names; if you have composite fields later, extend similarly to narrator parsing
    name = s.split(",")[0].strip()

    pid = make_slug_en(name)  
    return {"person_id": pid, "label_en": pid.replace("-", " "), "raw": s, "name_raw": name}

In [142]:
COLLECTOR_COLS = ["collector_1", "collector_2", "collector_3", "collector_4", "collector_5"]

df_kg = df_kg.copy()

def get_collectors_raw(row) -> list[str]:
    out = []
    for c in COLLECTOR_COLS:
        v = row.get(c)
        if pd.notna(v):
            vv = str(v).strip()
            if vv:
                out.append(vv)
    return out

df_kg["collectors_raw"] = df_kg.apply(get_collectors_raw, axis=1)
df_kg["collectors_norm"] = df_kg["collectors_raw"].apply(lambda lst: [parse_collector_cell(x) for x in lst])

df_kg["collector_person_ids"] = df_kg["collectors_norm"].apply(
    lambda lst: [d["person_id"] for d in lst if isinstance(d, dict) and d.get("person_id")]
)


In [143]:
all_collector_ids = sorted({pid for lst in df_kg["collector_person_ids"] for pid in (lst or [])})
coded_collectors = [pid for pid in all_collector_ids if pid.startswith("col")]
print("Coded collectors (col*):", len(coded_collectors))
print(coded_collectors[:100])

Coded collectors (col*): 44
['col001', 'col002', 'col003', 'col004', 'col005', 'col006', 'col007', 'col008', 'col009', 'col010', 'col011', 'col012', 'col013', 'col014', 'col015', 'col016', 'col017', 'col018', 'col019', 'col020', 'col021', 'col022', 'col023', 'col024', 'col025', 'col026', 'col027', 'col028', 'col029', 'col030', 'col031', 'col032', 'col033', 'col034', 'col035', 'col036', 'col037', 'col038', 'col039', 'col040', 'col041', 'col042', 'col043', 'col044']


In [144]:
df_kg["collector_person_ids"].head()

0         [fedor-koniaev]
1        [nina-valkevits]
2             [m-sokolov]
3     [aleksandr-kel-ner]
4    [ol-ga-gil-debrandt]
Name: collector_person_ids, dtype: object

In [145]:
def canon_id(x, merge_map: dict) -> str | None:
    """Map one id/slug to canonical via MERGE_MAP. Preserve nulls."""
    if x is None:
        return None
    if isinstance(x, float) and pd.isna(x):
        return None
    s = str(x).strip()
    if not s or s.lower() in {"nan", "none", "null"}:
        return None
    return merge_map.get(s, s)

def canon_id_list(x, merge_map: dict) -> list[str]:
    """
    Canonicalize a cell that can be:
      - list[str]
      - a single string
      - null
    Returns a de-duplicated list preserving order.
    """
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return []

    # If it's already a list/tuple/set, use it as-is
    if isinstance(x, (list, tuple, set)):
        items = list(x)
    else:
        # If it's a string, try to interpret as a single id.
        # (If you sometimes store stringified lists, adapt here.)
        items = [x]

    out = []
    seen = set()
    for it in items:
        cid = canon_id(it, merge_map)
        if cid and cid not in seen:
            out.append(cid)
            seen.add(cid)
    return out

# Apply to df_kg["collector_person_ids"] (list-valued column)
df_kg["collector_person_ids"] = df_kg["collector_person_ids"].apply(lambda v: canon_id_list(v, MERGE_MAP))

# Optional: also keep a pipe-joined string for quick inspection / exporting CSV
df_kg["collector_person_ids_str"] = df_kg["collector_person_ids"].apply(lambda lst: "|".join(lst) if lst else "")

## 9. Content description Normalizing

The field content_description is treated as a source-level archival note and is not rewritten, to preserve fidelity to the primary record and to support FAIR reproducibility. For export and UI robustness, a derived field content_description_clean is generated via loss-minimizing technical normalization only, without altering meaning:

- trim leading/trailing whitespace;
- map placeholder/empty tokens (nan, none, null, -) to null;
- collapse repeated whitespace and line breaks into a single space.

In the RDF export dcterms:description is populated from content_description_clean (when non-empty), while the unmodified content_description remains available in the tabular source as the authoritative “as-recorded” value.

In [146]:
def clean_free_text(s: str) -> str | None:
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return None
    s = str(s).strip()
    if not s or s.lower() in {"nan", "none", "null", "-"}:
        return None

    # remove square brackets (keep inner content)
    # e.g. "[foo] bar" -> "foo bar"
    s = re.sub(r"[\[\]]", "", s)

    # collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()

    return s if s else None

df_kg["content_description_clean"] = df_kg["content_description"].apply(clean_free_text)

In [147]:
df_kg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   tale_id                    112 non-null    string        
 1   collection                 112 non-null    string        
 2   volume_no                  112 non-null    Int64         
 3   source_ref                 112 non-null    object        
 4   rights_status              112 non-null    object        
 5   narrator                   110 non-null    object        
 6   collector_1                112 non-null    object        
 7   collector_2                21 non-null     object        
 8   collector_3                10 non-null     object        
 9   collector_4                8 non-null      object        
 10  collector_5                2 non-null      object        
 11  recording_parish           112 non-null    object        
 12  recordin

In [148]:
df_kg.tail(20)

,tale_id,collection,volume_no,source_ref,rights_status,narrator,collector_1,collector_2,collector_3,collector_4,...,narrator_age,narrator_birth_year,narrator_label_en,narrator_person_id,narrator_person_id_canon,collectors_raw,collectors_norm,collector_person_ids,collector_person_ids_str,content_description_clean
92,era_vene_13_37_5,era_vene,13,"ERA, Vene 13, 37/42 (5)",open,"Nimi: Аксинья, Vanus/Sünniaasta: 72 л., Märkus...",Зоя Жемчужина,NaN,NaN,NaN,...,72.0,NaN,Aksin'ia,aksin-ia,aksinia,[Зоя Жемчужина],"[{'person_id': 'zoia-zhemchuzhina', 'label_en'...",[zoia-zhemchuzhina],zoia-zhemchuzhina,"Женя брата оговаривает сестру, брат отрубает е..."
93,era_vene_13_43_6,era_vene,13,"ERA, Vene 13, 43/53 (6)",open,"Nimi: Мария Латкина, Vanus/Sünniaasta: 76 л.",Зоя Жемчужина,NaN,NaN,NaN,...,76.0,NaN,Mariia Latkina,mariia-latkina,mariia-latkina,[Зоя Жемчужина],"[{'person_id': 'zoia-zhemchuzhina', 'label_en'...",[zoia-zhemchuzhina],zoia-zhemchuzhina,Царевич и лисичка.
94,era_vene_13_73_9,era_vene,13,"ERA, Vene 13, 73/9 (9)",open,"Nimi: Мария Латкина, Vanus/Sünniaasta: 76 л.",Зоя Жемчужина,NaN,NaN,NaN,...,76.0,NaN,Mariia Latkina,mariia-latkina,mariia-latkina,[Зоя Жемчужина],"[{'person_id': 'zoia-zhemchuzhina', 'label_en'...",[zoia-zhemchuzhina],zoia-zhemchuzhina,Корова помогает дочке выполнять задания мачехи.
95,era_vene_13_90_12,era_vene,13,"ERA, Vene 13, 90/3 (12)",open,"Nimi: Анна Карпулистова, Vanus/Sünniaasta: 68 л.",Зоя Жемчужина,NaN,NaN,NaN,...,68.0,NaN,Anna Karpulistova,anna-karpulistova,anna-karpulistova,[Зоя Жемчужина],"[{'person_id': 'zoia-zhemchuzhina', 'label_en'...",[zoia-zhemchuzhina],zoia-zhemchuzhina,Снегурочка.
96,era_vene_13_93_13,era_vene,13,"ERA, Vene 13, 93/105 (13)",open,"Nimi: Анна Карпулистова, Vanus/Sünniaasta: 68 л.",Зоя Жемчужина,NaN,NaN,NaN,...,68.0,NaN,Anna Karpulistova,anna-karpulistova,anna-karpulistova,[Зоя Жемчужина],"[{'person_id': 'zoia-zhemchuzhina', 'label_en'...",[zoia-zhemchuzhina],zoia-zhemchuzhina,Превращённый в кобеля мужик.
97,era_vene_7_227_1,era_vene,7,"ERA, Vene 7, 227/9 (1)",open,"Nimi: Кира Трубенкова, Vanus/Sünniaasta: 1920",Кира Трубенкова,NaN,NaN,NaN,...,NaN,1920.0,Kira Trubenkova,kira-trubenkova,kira-trubenkova,[Кира Трубенкова],"[{'person_id': 'kira-trubenkova', 'label_en': ...",[kira-trubenkova],kira-trubenkova,Баба-яга. Старуха посылает патчерицу Ную к сес...
98,rkm_vene_1_82_47,rkm_vene,1,"RKM, Vene 1, 82/103 (47)",open,"Nimi: Васса Денисовна Гунина, Vanus/Sünniaasta...",Вера Воогла,NaN,NaN,NaN,...,NaN,1876.0,Vassa Denisovna Gunina,vassa-denisovna-gunina,vassa-denisovna-gunina,[Вера Воогла],"[{'person_id': 'vera-voogla', 'label_en': 'ver...",[vera-voogla],vera-voogla,Царевич Иван не может найти себе невесту. Три ...
99,rkm_vene_1_236_5,rkm_vene,1,"RKM, Vene 1, 236/44 (5)",open,"Nimi: Анна Терентьевна Гунина, Vanus/Sünniaast...",Вера Воогла,NaN,NaN,NaN,...,NaN,1896.0,Anna Terent'evna Gunina,anna-terent-evna-gunina,anna-terentevna-gunina,[Вера Воогла],"[{'person_id': 'vera-voogla', 'label_en': 'ver...",[vera-voogla],vera-voogla,Сказка о мертвой царевне и семи багатырях. по ...
100,tru_vkk_12_44_1,trü_vkk,12,"TRÜ, VKK 12, 44/67 (1)",restricted_anon,"nar014, Vanus/Sünniaasta: 1894, Märkus: неграм...",col026,col030,col038,col042,...,NaN,1894.0,nar014,nar014,nar014,"[col026, col030, col038, col042]","[{'person_id': 'col026', 'label_en': 'col026',...","[col026, col030, col038, col042]",col026|col030|col038|col042,"У купца три дочки и сын. Часто гуляют в саду, ..."
101,tru_vkk_32_17_3,trü_vkk,32,"TRÜ, VKK 32, 17/21 (3)",restricted_anon,"nar004, Vanus/Sünniaasta: 12 л., Märkus: услыш...",col015,NaN,NaN,NaN,...,12.0,NaN,nar004,nar004,nar004,[col015],"[{'person_id': 'col015', 'label_en': 'col015',...",[col015],col015,Мачеха отправляет падчерицу Лену к Бабе-Яги за...


## 10. Check missing values

In [149]:
df = df_kg.copy()
n = len(df)

# --- Define "coverage" signals (what will become missing triples in RDF) ---
# Narrator: missing -> no dcterms:contributor triple
miss_narrator = df["narrator_person_id_canon"].isna() | (df["narrator_person_id_canon"].astype(str).str.strip() == "")

# Recording place (settlement): missing -> no dcterms:spatial (place-level) triple (parish may still exist)
miss_recording_place = df["recording_place_english"].isna() | (df["recording_place_english"].astype(str).str.strip() == "")

# Type coverage: if all type_code_1..4 are missing, you won't emit dcterms:subject taleType triples
type_cols = [c for c in ["type_code_1", "type_code_2", "type_code_3", "type_code_4"] if c in df.columns]
miss_type_all = df[type_cols].isna().all(axis=1) if type_cols else pd.Series([True] * n, index=df.index)

# Description: missing -> no dcterms:description
desc_col = "content_description_clean" if "content_description_clean" in df.columns else "content_description"
miss_description = df[desc_col].isna() | (df[desc_col].astype(str).str.strip() == "")

# Volume: missing -> should never happen; if it does, you can't link tale -> volume reliably
miss_volume = df["volume_id"].isna() | (df["volume_id"].astype(str).str.strip() == "")

# --- Metrics table (counts + percentages) ---
metrics = [
    ("missing_narrator", int(miss_narrator.sum()), float(miss_narrator.mean() * 100)),
    ("missing_recording_place", int(miss_recording_place.sum()), float(miss_recording_place.mean() * 100)),
    ("missing_type_all", int(miss_type_all.sum()), float(miss_type_all.mean() * 100)),
    ("missing_description", int(miss_description.sum()), float(miss_description.mean() * 100)),
]

df_metrics = pd.DataFrame(metrics, columns=["metric", "count", "percent"]).sort_values("percent", ascending=False)
df_metrics["percent"] = df_metrics["percent"].round(2)

display(df_metrics)

# --- OPTIONAL: lists for thesis appendix / QA (which rows are missing what) ---
# (You can export these to CSV and commit as "expected outputs" / QA artifacts)
missing_lists = {
    "missing_narrator": df.loc[miss_narrator, ["tale_id", "narrator"]].copy(),
    "missing_recording_place": df.loc[miss_recording_place, ["tale_id", "recording_parish", "recording_place", "recording_place_english", "recording_parish_english"]].copy(),
    "missing_type_all": df.loc[miss_type_all, ["tale_id"] + type_cols].copy(),
    "missing_description": df.loc[miss_description, ["tale_id", "content_description"]].copy(),
}

# Preview (top 20 each)
for k, d in missing_lists.items():
    print(f"\n=== {k}: {len(d)} rows ===")
    display(d.head(20))


,metric,count,percent
2,missing_type_all,15,13.39
1,missing_recording_place,11,9.82
0,missing_narrator,2,1.79
3,missing_description,1,0.89



=== missing_narrator: 2 rows ===


,tale_id,narrator
1,era_vene_5_167_9,NaN
109,tru_vkk_25_120_23,NaN



=== missing_recording_place: 11 rows ===


,tale_id,recording_parish,recording_place,recording_place_english,recording_parish_english
3,era_vene_7_125_1,Tallinna linn,NaN,NaN,Tallinn
4,rkm_vene_3_257_116,Tartu linn,NaN,NaN,Tartu
10,era_vene_3_292_6,Tartu linn,NaN,NaN,Tartu
11,era_vene_7_75_1,Tallinna linn,NaN,NaN,Tallinn
26,era_vene_7_ 95_1,Tallinna linn,NaN,NaN,Tallinn
33,era_vene_14_451_7,Tartu linn,NaN,NaN,Tartu
38,rkm_vene_3_257_117,Tartu linn,NaN,NaN,Tartu
62,era_vene_7_89_1,Tallinna linn,NaN,NaN,Tallinn
63,era_vene_7_71_1,Tallinna linn,NaN,NaN,Tallinn
64,era_vene_7_221_1,Tallinna linn,NaN,NaN,Tallinn



=== missing_type_all: 15 rows ===


,tale_id,type_code_1,type_code_2,type_code_3,type_code_4
0,era_vene_2_605_4,<NA>,<NA>,<NA>,<NA>
1,era_vene_5_167_9,<NA>,<NA>,<NA>,<NA>
2,era_vene_6_379_6,<NA>,<NA>,<NA>,<NA>
3,era_vene_7_125_1,<NA>,<NA>,<NA>,<NA>
4,rkm_vene_3_257_116,<NA>,<NA>,<NA>,<NA>
5,tru_vkk_52_85_6,<NA>,<NA>,<NA>,<NA>
6,tru_vkk_38_69_33,<NA>,<NA>,<NA>,<NA>
7,tru_vkk_41_94_45,<NA>,<NA>,<NA>,<NA>
8,tru_vkk_32_22_4,<NA>,<NA>,<NA>,<NA>
9,era_vene_1_521_1,<NA>,<NA>,<NA>,<NA>



=== missing_description: 1 rows ===


,tale_id,content_description
90,era_vene_13_345_29,NaN


## 11.  Coverage and completeness

The current corpus slice (N = 112 tales) shows **good baseline metadata coverage** for agents, places, and descriptions, with the main incompleteness concentrated in **tale-type assignment**.

| Metric | Count | Share | Interpretation for the KG |
|---|---:|---:|---|
| `missing_type_all` | 15 | 13.39% | These tales have **no type codes in any of `type_code_1..4`**, therefore the KG will not emit `dcterms:subject` links to tale-type concepts for them. This is treated as a **knowledge-quality / coverage signal**, not an ETL error. |
| `missing_recording_place` | 11 | 9.82% | The settlement-level `recording_place` is missing. In the current data, this often corresponds to cases where only **parish/municipality is recorded** (e.g., “Tallinna linn”, “Tartu linn”). We handle this with a **controlled fallback** (parish-as-place), explicitly flagged, to improve spatial interoperability without fabricating new information. |
| `missing_narrator` | 2 | 1.79% | Narrator is absent for a small minority of records; the KG simply omits `dcterms:contributor` for those tales. This supports transparent provenance and avoids inventing agent identities. |
| `missing_description` | 1 | 0.89% | A single tale has no archival `content_description`; the KG omits `dcterms:description` for that record. |

#### What this demonstrates 

- **Truthfulness to source granularity:** Missing values are preserved (no imputation) and represented in RDF by the **absence of triples**, which cleanly distinguishes “not recorded” from “recorded as unknown”.
- **Data governance via measurable quality signals:** Coverage metrics are computed and versioned, enabling **regression checks across releases** (e.g., type coverage improving after additional mapping work).

Overall coverage is high for core provenance and descriptive fields; the principal limitation is incomplete tale-type coverage (13.39%), which is intentionally surfaced as a knowledge-quality indicator and a target for future enrichment.

## 12. Clean dataset saving


In [150]:
OUT_DIR = PROJECT_ROOT / "data" / "processed"

df_kg.to_parquet(OUT_DIR / "corpus_a_for_kg.parquet", index=False)

df_kg.to_csv(OUT_DIR / "corpus_a_for_kg.csv", index=False, encoding="utf-8")

print("Saved")

Saved


#### Canonical table as the single source of truth

After data cleaning, normalization, and schema alignment, the project produces a canonical table: a curated, column-stable tabular representation where each row corresponds to a single tale record and each field follows explicit datatype and controlled-vocabulary commitments.

The final dataset is defined as the materialization of this canonical table. All downstream exports (Turtle, CSV distributions) are derived from the canonical table and must not introduce additional semantics beyond the documented transformation rules.